# Hackathon

Some utilities

## Import Utils

In [2]:
import keras
from keras.metrics import top_k_categorical_accuracy
import h5py as h5
import numpy as np
import pandas as pd
import tables

PATH_DATA = '/notebooks/data/full.h5'

In [3]:
def get_idxs(h5_path):
    f = h5.File(h5_path)
    return range(len(f['S2']))

def shuffle_idx(sample_idxs):
    return list(np.random.permutation(sample_idxs))

def split_train_val(sample_idxs, proportion):
    n_samples = len(sample_idxs)
    return sample_idxs[:int((1.-proportion)*n_samples)], sample_idxs[int((1.-proportion)*n_samples):]

def get_batch_count(idxs, batch_size):
    batch_count = int(len(idxs)//batch_size)
    remained_samples = len(idxs)%batch_size
    if remained_samples > 0:
        batch_count += 1

    return batch_count

In [4]:
def generator(h5_path, batch_size, idxs):
    f = h5.File(h5_path, 'r')
    while True : 
        idxs = shuffle_idx(idxs)
        batch_count = get_batch_count(idxs, batch_size)
        for b in range(batch_count):
            batch_idx = idxs[b*batch_size]
            X = f['S2'][batch_idx:batch_idx+batch_size, :,:,:]
            Y = f['TOP_LANDCOVER'][batch_idx:batch_idx+batch_size, :]
            yield np.array(X), keras.utils.np_utils.to_categorical(np.array(Y), 23)

In [37]:
idxs = get_idxs(PATH_DATA)
shuffled_idxs = shuffle_idx(idxs)
train_idxs, val_idxs = split_train_val(shuffled_idxs, 0.2)

print(len(idxs))

53606


In [5]:
label = h5.File(PATH_DATA)['TOP_LANDCOVER']
images = h5.File(PATH_DATA)['S2']

In [ ]:
counts = np.zeros(23)
lbl = np.zeros(len(label))

for idx, value in enumerate(label):
    counts[int(value)] += 1
    lbl[idx] = value

In [ ]:
import matplotlib.pyplot as plt

plt.bar(np.arange(len(counts)), counts)
plt.show()

In [ ]:
##DataSet Full
dsetfull = h5.File(PATH_DATA)['TOP_LANDCOVER']
labelsfull = dset[0:].astype(int)
data4Dfull = h5.File(PATH_DATA)['S2']

In [ ]:
##DataSet eigthies
dset80 = h5.File(PATH_80s)['TOP_LANDCOVER']
labels80 = dset80[0:].astype(int)
data4D80 = h5.File(PATH_80s)['S2']

In [ ]:
##PATH Predict
data4Dpredict = h5.File(PATH_PREDICT_WITHOUT_GT)['S2']

In [ ]:
##Traitement des labels : 23 -> 14
unique, counts = np.unique(labelsfull, return_counts=True)
unique = pd.DataFrame(unique)
counts = pd.DataFrame(counts)
prop = pd.DataFrame(counts/18698240*100)

labelsCountsfull = pd.concat([unique, counts,prop], axis=1, join_axes=[unique.index])
labelsCountsfull.columns = ['label','count','prop']
labelsCountsfull
minProp = 0.02
labelsCountsFiltered = labelsCountsfull[labelsCountsfull['prop']>minProp]
labelsCountsFiltered = labelsCountsFiltered.reset_index()


In [ ]:
equilibre = np.zeros((14,3830))

for i in range(len(labelsCountsFiltered)):
    current = labelsCountsFiltered.iloc[i,1]
    print(current)
    equilibre[i][0] = current
    equilibre[i][1:] = np.where(labels == current)[0][:3829]


In [ ]:
Equidxs = np.zeros(14*3829)
for i in range(14):
    for j in range(3829):
        Equidxs[i*3829+j] = equilibre[i,j+1]
        
Equidxs = np.sort(Equidxs)
Equidxs

In [ ]:
##DataSet Equilibré
dset = h5.File(PATH_DATA)['TOP_LANDCOVER'][Equidxs,:]
labels = dset[0:].astype(int)
data4D = h5.File(PATH_DATA)['S2'][Equidxs,:,:,:]

In [ ]:
plt.hist(labels)